In [ ]:
!cp '/content/drive/My Drive/Projects/Nishikita Competition/data.zip' /content/
!unzip /content/data.zip
!unzip /content/train.zip
!unzip /content/test.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/n_01180.jpg       
  inflating: train/n_03420.jpg       
  inflating: train/n_00970.jpg       
  inflating: train/n_03709.jpg       
  inflating: train/n_01967.jpg       
  inflating: train/n_03507.jpg       
  inflating: train/n_00854.jpg       
  inflating: train/n_04549.jpg       
  inflating: train/n_00470.jpg       
  inflating: train/n_02222.jpg       
  inflating: train/n_00305.jpg       
  inflating: train/n_03576.jpg       
  inflating: train/n_00526.jpg       
  inflating: train/n_00708.jpg       
  inflating: train/n_03032.jpg       
  inflating: train/n_04688.jpg       
  inflating: train/n_00516.jpg       
  inflating: train/n_03807.jpg       
  inflating: train/n_02134.jpg       
  inflating: train/n_02929.jpg       
  inflating: train/n_04367.jpg       
  inflating: train/n_05162.jpg       
  inflating: train/n_00272.jpg       
  inflating: train/n_05097.jpg       
  inflating: train/n_01494.jpg       

In [ ]:
!pip install -U --pre efficientnet
!pip install tf2cv

     |████████████████████████████████| 348kB 4.7MB/s 


In [ ]:
import numpy as np 
import tensorflow 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D,Dense,concatenate,Input
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import ResNet101,ResNet50,ResNet152
from tensorflow.keras.utils import Sequence 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import *
import efficientnet.tfkeras as efn
import math
import tf2cv
from tf2cv.model_provider import get_model as tf2cv_get_model
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate,Cutout
)
#@title Imports
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import time
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
from tqdm import tqdm

In [ ]:
data_dir='/content/'
classes_map={
    0: 'Male-Aristocrat', 1: 'Male-Samurai', 2: 'Male-Animal', 3: 'Male-Common People',
4: 'Female-Aristocrat', 5: 'Female-Samurai', 6: 'Female-Animal' , 7: 'Woman-common people'
}

img_size=256
batch_size=64
epochs=10
n_classes=len(classes_map.items())
lr=1e-2
model_dir='/content/drive/My Drive/Projects/Nishikita Competition/Models/bitransferresnet.hdf5'
model_dirs=['/content/drive/My Drive/Projects/Nishikita Competition/Models/definitive.hdf5','/content/drive/My Drive/Projects/Nishikita Competition/Models/bitransferresnet.hdf5','/content/drive/My Drive/Projects/Nishikita Competition/Models/bitresnet101.hdf5','/content/drive/My Drive/Projects/Nishikita Competition/Models/bitresnet50*3.hdf5','/content/drive/My Drive/Projects/Nishikita Competition/Models/bitresnet101*3.hdf5']

In [ ]:
train_df=pd.read_csv(data_dir+'sample_submission.csv')
train_df['image']=train_df['image'].apply(lambda x:data_dir+'test/'+x)
train_df.head()

,image,gender_status
0,/content/test/n_00000.jpg,0
1,/content/test/n_00004.jpg,0
2,/content/test/n_00009.jpg,0
3,/content/test/n_00026.jpg,0
4,/content/test/n_00030.jpg,0


In [ ]:
from tensorflow.keras.utils import Sequence 

class AnimeData(Sequence):
  def __init__(self,x_set,batch_size=batch_size):
    self.x=x_set
    self.batch_size=batch_size 
  def __getitem__(self,step):
    start=step*self.batch_size 
    end=(step+1)*self.batch_size 
    x_batch=self.x[start:end]
    x_batch=self.data_gen(x_batch)
    return x_batch/255.0
  
  def data_gen(self,batch_x):
    x_batch=np.empty((batch_x.shape[0],img_size,img_size,3))
    for i in range(batch_x.shape[0]):
      x=cv2.imread(batch_x[i])
      x_batch[i]=x
    return np.array(x_batch)
  
  def __len__(self):
    return math.ceil(len(self.x) / self.batch_size)

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
#All model building functions defined here. 
def get_efficientnet(shape=(img_size,img_size,3),weights='noisy-student',trainable_base=False,name='b7'):
  if name=='b7':
    base=efn.EfficientNetB7(weights=weights,input_shape=shape,include_top=False)
  elif name=='b6':
    base=efn.EfficientNetB6(weights=weights,input_shape=shape,include_top=False)
  elif name=='b5':
    base=efn.EfficientNetB5(weights=weights,input_shape=shape,include_top=False)
  elif name=='b4':
    base=efn.EfficientNetB4(weights=weights,input_shape=shape,include_top=False)
  elif name=='b3':
    base=efn.EfficientNetB3(weights=weights,input_shape=shape,include_top=False)
  elif name=='b2':
    base=efn.EfficientNetB2(weights=weights,input_shape=shape,include_top=False)
  elif name=='b1':
    base=efn.EfficientNetB1(weights=weights,input_shape=shape,include_top=False)
  elif name=='l2':
    base=efn.EfficientNetL2(weights=weights,input_shape=shape,include_top=False)
  base.trainable=trainable_base
  avg_pool=GlobalAveragePooling2D()(base.output)
  #max_pool=GlobalMaxPooling2D()(base.output)
  #out=concatenate([avg_pool,max_pool])
  #avg_pool=Dense(1024,activation='relu')(avg_pool)
  out=Dense(n_classes,activation='softmax')(avg_pool)
  model=Model(inputs=[base.input],outputs=[out])
  return model



def get_tf2cv_model(input_shape=(img_size,img_size,3),trainable_base=False):
  net=tf2cv_get_model('senet154',pretrained=True)
  net.trainable=trainable_base
  inp=Input(input_shape)
  model_out=net.layers[-2](inp)
  model_out=GlobalAveragePooling2D()(model_out)
  model_out=Dense(n_classes,activation='softmax')(model_out)
  model=Model(inputs=[inp],outputs=[model_out])
  return model


def get_bitresnet(input_shape=(img_size,img_size,3),trainable_base=False,variant='50'):
  # Load model into KerasLayer
  inp=Input(shape=input_shape)
  if variant=='50':
    model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
  elif variant=='101':
    model_url="https://tfhub.dev/google/bit/m-r101x1/1"
  elif variant=='50*3':
    model_url="https://tfhub.dev/google/bit/m-r50x3/1"
  elif variant=='101*3':
    model_url="https://tfhub.dev/google/bit/s-r101x3/1"
  module = hub.KerasLayer(model_url)
  model_out=module(inp)
  model_out=Dense(n_classes,activation='softmax')(model_out)
  model=Model(inputs=[inp],outputs=[model_out])
  return model

In [ ]:
#Averaging Ensemble (A Little crude)
test_gen=AnimeData(train_df.image.values,batch_size=1)
prediction_full=np.empty((len(train_df),n_classes,len(model_dirs)))
for i,path in tqdm(enumerate(model_dirs)):
  if i==0:
    variant='b7'
  elif i==1:
    variant='50'
  elif i==2:
    variant='101'
  elif i==3:
    variant='50*3'
  elif i==4:
    variant='101*3'
  if variant in [f'b{i+1}' for i in range(7)]:
    model=get_efficientnet(name=variant)
  else:
    model=get_bitresnet(variant=variant)
  model.load_weights(path)
  preds=model.predict(test_gen,verbose=1)
  prediction_full[:,:,i]=preds 

predictions_full=np.average(prediction_full,axis=-1)
preds=predictions_full.argmax(axis=-1)
sample=pd.read_csv('sample_submission.csv')
train_df['image']=sample['image']
train_df['gender_status']=preds
train_df.to_csv('submit.csv',index=False)